In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
from pprint import pprint
import torch
import sys
import yaml
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from Utils import TransformerSampler, ModelConfig, GenerationConfig
from model import Model
from statedict_mapping import convert_model_weights
torch.manual_seed(1337)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cuda


In [68]:
sampler = TransformerSampler()
model = AutoModelForCausalLM.from_pretrained("/home/ubuntu/MechInter/1LayerModel/Models/TinyStories-1L-21M").to(device)
tokenizer = AutoTokenizer.from_pretrained("/home/ubuntu/MechInter/1LayerModel/Models/TinyStories-1L-21M")
tokenizer.pad_token = tokenizer.eos_token

with open('OneLM.yaml', 'r') as f:
    config_dict = yaml.safe_load(f)
    model_config = ModelConfig(config_dict)

modified_state_dict = convert_model_weights(model_config, model)
model = Model(model_config).to(device)
model.load_state_dict(modified_state_dict)

<All keys matched successfully>

In [63]:
model

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(2048, 1024)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): GPTNeoBlock(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=False)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
          (c_proj): Linear(

In [66]:
for key, val in model.state_dict().items():
    print(key, val.shape)

transformer.wte.weight torch.Size([50257, 1024])
transformer.wpe.weight torch.Size([2048, 1024])
transformer.h.0.ln_1.weight torch.Size([1024])
transformer.h.0.ln_1.bias torch.Size([1024])
transformer.h.0.attn.attention.k_proj.weight torch.Size([1024, 1024])
transformer.h.0.attn.attention.v_proj.weight torch.Size([1024, 1024])
transformer.h.0.attn.attention.q_proj.weight torch.Size([1024, 1024])
transformer.h.0.attn.attention.out_proj.weight torch.Size([1024, 1024])
transformer.h.0.attn.attention.out_proj.bias torch.Size([1024])
transformer.h.0.ln_2.weight torch.Size([1024])
transformer.h.0.ln_2.bias torch.Size([1024])
transformer.h.0.mlp.c_fc.weight torch.Size([4096, 1024])
transformer.h.0.mlp.c_fc.bias torch.Size([4096])
transformer.h.0.mlp.c_proj.weight torch.Size([1024, 4096])
transformer.h.0.mlp.c_proj.bias torch.Size([1024])
transformer.ln_f.weight torch.Size([1024])
transformer.ln_f.bias torch.Size([1024])
lm_head.weight torch.Size([50257, 1024])


In [69]:
# prompts = [
#     'Once upon a time',
#     'In a galaxy far, far away',
#     'The quick brown fox',
# ]

prompts = 'hi there'

tokens = tokenizer(prompts, return_tensors='pt', padding=True, padding_side='left', truncation=True).to(device='cuda')
input_ids = tokens['input_ids']
attention_mask = tokens['attention_mask']

for _ in range(20):
    logits = model(input_ids = input_ids, attention_mask = attention_mask)
    next_token = sampler.decode(logits, temperature=0.8).to(device='cuda')
    print(tokenizer.batch_decode(next_token))
    input_ids = torch.cat([input_ids, next_token.unsqueeze(-1)], dim=1)
    attention_mask = torch.cat([attention_mask, torch.ones((attention_mask.shape[0], 1), dtype=attention_mask.dtype, device='cuda')], dim=-1)
# print(tokenizer.batch_decode(input_ids))


[' chase']
[' chase']
['.']
[' Sam']
[' Sam']
[' Tim']
[' Tim']
[' Tim']
[' Tim']
[' Tim']
[' Tim']
[' Tim']
[' Tim']
[' Tim']
[' Tim']
[' Tim']
[' Tim']
[' Tim']
[' Tim']
[' Tim']
